<a href="https://colab.research.google.com/github/samuelhtampubolon/SDPM2025/blob/main/Pinjaman_Bank_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%pip install openpyxl
import pandas as pd
import numpy as np

# 1. MEMUAT DATA
# Ganti dengan path file Anda jika berbeda
file_path = '/content/Pinjaman_Bank_A.xlsx'
df = pd.read_excel(file_path)
print("--- DATA AWAL ---")
print(df.info())

--- DATA AWAL ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID_Peminjam         100 non-null    object        
 1   Tanggal_Aplikasi    100 non-null    datetime64[ns]
 2   Nama_Peminjam       100 non-null    object        
 3   Jumlah_Pinjaman     96 non-null     object        
 4   Jangka_Waktu_Bulan  100 non-null    object        
 5   Suku_Bunga_Tahunan  99 non-null     object        
 6   Tujuan_Pinjaman     100 non-null    object        
 7   Pendapatan_Bulanan  99 non-null     object        
 8   Status_Persetujuan  100 non-null    object        
 9   Skor_Kredit         100 non-null    object        
 10  Unnamed: 10         1 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 8.7+ KB
None


In [ ]:
print("--- DATA AWAL ---")
print(df.info())# 2. INSPEKSI AWAL
print("--- DATA AWAL ---")
print(df.info())
print("\nJumlah nilai yang hilang per kolom:")
print(df.isnull().sum())
print("\nContoh 5 baris data kotor:")
print(df.head())
print("\nJumlah baris duplikat:", df.duplicated().sum())

--- DATA AWAL ---
<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 99
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID_Peminjam         98 non-null     object        
 1   Tanggal_Aplikasi    98 non-null     datetime64[ns]
 2   Nama_Peminjam       98 non-null     object        
 3   Jumlah_Pinjaman     98 non-null     int64         
 4   Jangka_Waktu_Bulan  98 non-null     int64         
 5   Suku_Bunga_Tahunan  98 non-null     float64       
 6   Tujuan_Pinjaman     98 non-null     object        
 7   Pendapatan_Bulanan  98 non-null     int64         
 8   Status_Persetujuan  98 non-null     object        
 9   Skor_Kredit         98 non-null     int64         
 10  Unnamed: 10         1 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(4)
memory usage: 9.2+ KB
None

Jumlah nilai yang hilang per kolom:
ID_Peminjam            0
Tanggal_

In [ ]:
# 3. PROSES PEMBERSIHAN DATA
# 3.1. Menangani Data Duplikat
df.drop_duplicates(inplace=True)

# 3.2. Membersihkan Kolom 'ID_Peminjam' (tidak ada masalah serius, hanya sebagai contoh)
# df['ID_Peminjam'] = df['ID_Peminjam'].str.strip() # Menghapus spasi jika ada

# 3.3. Membersihkan Kolom 'Nama_Peminjam'
df['Nama_Peminjam'] = df['Nama_Peminjam'].str.strip().str.title()

# 3.4. Membersihkan dan Mengonversi 'Tanggal_Aplikasi' ke Tipe Datetime
# errors='coerce' akan mengubah tanggal yang tidak valid menjadi NaT (Not a Time)
df['Tanggal_Aplikasi'] = pd.to_datetime(df['Tanggal_Aplikasi'], errors='coerce')

# 3.5. Membersihkan dan Mengonversi 'Jumlah_Pinjaman' ke Tipe Numerik (Integer)
# Menggunakan regex untuk menghapus semua karakter non-numerik
df['Jumlah_Pinjaman'] = df['Jumlah_Pinjaman'].astype(str).str.replace(r'[^\d]', '', regex=True)
# Mengganti 'jt' dengan '000000'
df['Jumlah_Pinjaman'] = df['Jumlah_Pinjaman'].str.replace('jt', '000000', regex=False)
# Konversi ke numerik, error diubah jadi NaN (nilai hilang)
df['Jumlah_Pinjaman'] = pd.to_numeric(df['Jumlah_Pinjaman'], errors='coerce')

# 3.6. Membersihkan dan Mengonversi 'Jangka_Waktu_Bulan'
# Fungsi untuk konversi 'tahun' ke bulan
def konversi_ke_bulan(waktu):
    waktu = str(waktu).lower()
    if 'tahun' in waktu:
        # Ambil angka dari string, kalikan 12
        angka = int(''.join(filter(str.isdigit, waktu)))
        return angka * 12
    else:
        # Jika bukan 'tahun', coba konversi langsung ke angka
        return pd.to_numeric(waktu, errors='coerce')

df['Jangka_Waktu_Bulan'] = df['Jangka_Waktu_Bulan'].apply(konversi_ke_bulan)
# Mengubah nilai negatif menjadi positif (asumsi salah input)
df['Jangka_Waktu_Bulan'] = df['Jangka_Waktu_Bulan'].abs()

# 3.7. Membersihkan dan Mengonversi 'Suku_Bunga_Tahunan'
df['Suku_Bunga_Tahunan'] = df['Suku_Bunga_Tahunan'].astype(str).str.replace('%', '')
df['Suku_Bunga_Tahunan'] = pd.to_numeric(df['Suku_Bunga_Tahunan'], errors='coerce')

# 3.8. Menstandarisasi Kategori 'Tujuan_Pinjaman'
df['Tujuan_Pinjaman'] = df['Tujuan_Pinjaman'].str.strip().str.lower()
peta_tujuan = {
    'kredit rumah': 'Properti', 'kpr': 'Properti', 'beli rumah': 'Properti',
    'beli properti': 'Properti', 'renovasi rumah': 'Renovasi',
    'modal usaha': 'Usaha', 'investasi bisnis': 'Usaha', 'modal ventura': 'Usaha', 'start-up': 'Usaha',
    'konsumtif': 'Konsumtif', 'liburan': 'Konsumtif', 'pernikahan': 'Konsumtif',
    'darurat': 'Kebutuhan Mendesak', 'biaya medis': 'Kebutuhan Mendesak',
    'beli mobil': 'Kendaraan'
}
# Menggunakan .get(x, x) agar nilai yang tidak ada di peta tetap sama
df['Tujuan_Pinjaman'] = df['Tujuan_Pinjaman'].apply(lambda x: peta_tujuan.get(x, x)).str.title()


# 3.9. Menstandarisasi Kategori 'Status_Persetujuan'
peta_status = {
    '1': 'Disetujui', 'Approved': 'Disetujui',
    '0': 'Ditolak', 'Rejected': 'Ditolak'
}
df['Status_Persetujuan'] = df['Status_Persetujuan'].replace(peta_status)

# 3.10. Menangani Outlier dan Membersihkan 'Skor_Kredit'
# Skor kredit wajar antara 300-850. Nilai di luar itu dianggap error/NaN
df['Skor_Kredit'] = pd.to_numeric(df['Skor_Kredit'], errors='coerce')
df.loc[(df['Skor_Kredit'] < 300) | (df['Skor_Kredit'] > 850), 'Skor_Kredit'] = np.nan

In [ ]:
# 4. MENGISI NILAI YANG HILANG (IMPUTASI)
# Strategi:
# - Jumlah_Pinjaman & Pendapatan_Bulanan: isi dengan median (lebih tahan terhadap outlier)
# - Jangka_Waktu_Bulan, Suku_Bunga_Tahunan, Skor_Kredit: isi dengan mean (rata-rata)
# - Tanggal_Aplikasi: isi dengan ffill (forward fill) or bfill (backward fill)
# - Tujuan_Pinjaman: isi dengan modus ('Unknown')

# Pertama, bersihkan dulu 'Pendapatan_Bulanan' seperti 'Jumlah_Pinjaman'
df['Pendapatan_Bulanan'] = df['Pendapatan_Bulanan'].astype(str).str.replace(r'[^\d]', '', regex=True)
df['Pendapatan_Bulanan'] = pd.to_numeric(df['Pendapatan_Bulanan'], errors='coerce')

# Lakukan imputasi
for col in ['Jumlah_Pinjaman', 'Pendapatan_Bulanan']:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)

for col in ['Jangka_Waktu_Bulan', 'Suku_Bunga_Tahunan', 'Skor_Kredit']:
    mean_val = df[col].mean()
    df[col].fillna(mean_val, inplace=True)

df['Tanggal_Aplikasi'].fillna(method='ffill', inplace=True)
df['Tujuan_Pinjaman'].fillna('Unknown', inplace=True)
df.dropna(subset=['Nama_Peminjam'], inplace=True) # Hapus baris jika nama peminjam kosong

/tmp/ipython-input-3837993414.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_val, inplace=True)
/tmp/ipython-input-3837993414.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

In [ ]:
# 5. MENGUBAH TIPE DATA FINAL
df = df.astype({
    'Jumlah_Pinjaman': 'int64',
    'Jangka_Waktu_Bulan': 'int64',
    'Pendapatan_Bulanan': 'int64',
    'Skor_Kredit': 'int64'
})

In [ ]:
# 6. VERIFIKASI AKHIR
print("\n\n--- DATA SETELAH DIBERSIHKAN ---")
print(df.info())
print("\nJumlah nilai yang hilang per kolom (setelah pembersihan):")
print(df.isnull().sum())
print("\nContoh 5 baris data bersih:")
print(df.head())
print("\nDeskripsi Statistik Data Bersih:")
print(df.describe())



--- DATA SETELAH DIBERSIHKAN ---
<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 99
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   ID_Peminjam         98 non-null     object        
 1   Tanggal_Aplikasi    98 non-null     datetime64[ns]
 2   Nama_Peminjam       98 non-null     object        
 3   Jumlah_Pinjaman     98 non-null     int64         
 4   Jangka_Waktu_Bulan  98 non-null     int64         
 5   Suku_Bunga_Tahunan  98 non-null     float64       
 6   Tujuan_Pinjaman     98 non-null     object        
 7   Pendapatan_Bulanan  98 non-null     int64         
 8   Status_Persetujuan  98 non-null     object        
 9   Skor_Kredit         98 non-null     int64         
 10  Unnamed: 10         1 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(4)
memory usage: 9.2+ KB
None

Jumlah nilai yang hilang per kolom (setelah pembersi